In [1]:
%cd workspace/
import os
# Set the correct MongoDB URI for FiftyOne
os.environ["FIFTYONE_DATABASE_URI"] = "mongodb://172.19.0.2:27017"
# Verify the change
print(os.environ["FIFTYONE_DATABASE_URI"])

# imports
from autodistill_detic import DETIC
from autodistill.detection import CaptionOntology
import fiftyone as fo



/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspace
mongodb://172.19.0.2:27017


In [2]:
# define an ontology to map class names to our DETIC prompt
# the ontology dictionary has the format {caption: class}
# where caption is the prompt sent to the base model, and class is the label that will
# be saved for that caption in the generated annotations
# then, load the model
base_model = DETIC(
    ontology=CaptionOntology(
        {
            "person": "person",
        }
    )
)
base_model.label("/home/chelo/datastore/test_images", extension=".jpg")

100%|███████████████████████████████████████| 338M/338M [00:31<00:00, 11.2MiB/s]


Loading pretrained CLIP


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)


[10/12 22:45:14 fvcore.common.checkpoint]: [Checkpointer] Loading from models/Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth ...


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Labeling /home/chelo/datastore/test_images/images.jpg: 100%|██████████| 8/8 [00:05<00:00,  1.47it/s]                                  

Labeled dataset created - ready for distillation.



Passing a `Dict[str, np.ndarray]` into `DetectionDataset` is deprecated and will be removed in `supervision-0.26.0`. Use a list of paths `List[str]` instead.


In [41]:
# Path to the folder containing images and YOLO annotations
dataset_dir = "/home/chelo/datastore/test_images_labeled/"
yaml_path = dataset_dir + "data.yaml"  # Path to dataset.yaml
# name of the dataset
dataset_name = "test"

# Define the classes
classes = ["person"]

if dataset_name in fo.list_datasets():
    # load the dataset if it already exists
    print("Loading dataset")
    dataset = fo.load_dataset("test")
    print("Dataset loaded")
else:
    print("Creating dataset")
    # Load the dataset into FiftyOne
    
    

    # The splits to load
    splits = ["train", "val"]
    
    dataset = fo.Dataset(dataset_name)
    for split in splits:
        dataset.add_dir(
            dataset_dir=dataset_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            yaml_path=yaml_path, 
            split=split,
            tags=split,
    )
    dataset.persistent = True
    print("Dataset created")
    
# Check how many samples are in the train and val splits
train_view = dataset.match_tags("train")
val_view = dataset.match_tags("val")
print(f"Train samples: {len(train_view)}")
print(f"Validation samples: {len(val_view)}")


Loading dataset
Dataset loaded
Train samples: 6
Validation samples: 2


In [42]:
# Launch the FiftyOne App to visualize
session = fo.launch_app(dataset, port=5152, auto=False)
# curl http://localhost:5152

Session launched. Run `session.show()` to open the App in a cell output.
